<a href="https://colab.research.google.com/github/TanbinTanny5669/Architects-expo/blob/main/raw/baseline/JP24B01-Baseline_various_mixes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You might need to install this on your system:

apt-get install python3-opencv git

In [1]:
import os

if not os.path.isdir('k'):
  !git clone https://github.com/joaopauloschuler/k-neural-api.git k
else:
  !cd k && git pull

!cd k && pip install .

Cloning into 'k'...
remote: Enumerating objects: 1838, done.
remote: Counting objects: 100% (250/250), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 1838 (delta 158), reused 171 (delta 88), pack-reused 1588
Receiving objects: 100% (1838/1838), 15.70 MiB | 19.80 MiB/s, done.
Resolving deltas: 100% (1271/1271), done.
Processing /content/k
  Preparing metadata (setup.py) ... done
  Created wheel for cai: filename=cai-0.1.7-py3-none-any.whl size=61388 sha256=74e85905b8d490b12f66a4a3e9fa89d80f05ebe0bcf742fd43d0053ee13178f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-pwsrmcmw/wheels/80/61/f5/947bedc7e497038def7d1381fb65d37bd126a80e010114b8f1
Successfully built cai


In [2]:
!ls -l

total 8
drwxr-xr-x 8 root root 4096 Mar  4 10:38 k
drwxr-xr-x 1 root root 4096 Feb 28 14:27 sample_data


In [3]:
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

import skimage
print('skimage version',  skimage.__version__)
import tensorflow as tf
print(tf.__version__)
import sys
import cai
import cai.datasets
import cai.densenet
import cai.util
import cai.models

Python version
3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Version info.
sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0)
skimage version 0.19.3
2.15.0


In [ ]:
# Download from https://data.mendeley.com/datasets/tywbtsjrjv/1
url_zip_file="https://data.mendeley.com/datasets/tywbtsjrjv/1/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/Plant_leaf_diseases_dataset_without_augmentation.zip?dl=1"
local_zip_file="plant_leaf.zip"
expected_folder_name="plant_leaf"
Verbose=True
cai.datasets.download_zip_and_extract(
    url_zip_file=url_zip_file, local_zip_file=local_zip_file,
    expected_folder_name=expected_folder_name, Verbose=Verbose)

Downloading:  https://data.mendeley.com/datasets/tywbtsjrjv/1/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/Plant_leaf_diseases_dataset_without_augmentation.zip?dl=1  to  plant_leaf.zip
Decompressing into:  plant_leaf


In [45]:
import random
import os
import multiprocessing
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

from keras.utils import to_categorical
from keras.optimizers import SGD

from keras.layers import *
from keras.models import Model
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
import os

# Define the directory path
dataset_dir = "plant_leaf/Plant_leave_diseases_dataset_without_augmentation/"

# Check if the directory exists before attempting to remove it
if os.path.exists(dataset_dir):
    # If it exists, remove the directory recursively
    os.system('rm -r ' + dataset_dir)
else:
    print("The directory does not exist.")

# Now try to list the contents of the directory
try:
    print(os.listdir(dataset_dir))
except FileNotFoundError:
    print("The directory does not exist.")

The directory does not exist.
The directory does not exist.


In [38]:
def load_plantvillage(seed=None, root_dir=None):
  if root_dir == None:
    print("define where the plantvillage folder is.")
    return
  random.seed(seed)
  def read_from_paths(paths):
    x=[]
    for path in paths:
      img = load_img(path, target_size=(224,224))
      img = img_to_array(img, dtype='float16')
      x.append(img)
    return x

  classes = os.listdir(root_dir)
  classes = sorted(classes)

  train_path = []
  val_path = []
  test_path = []

  train_x,train_y = [],[]
  val_x,val_y = [],[]
  test_x,test_y =[],[]

  #read path and categorize to three groups , 6,2,2
  for i,_class in enumerate(classes):
      paths = glob.glob(os.path.join(root_dir,_class,"*"))
      paths = [n for n in paths if n.endswith(".JPG") or n.endswith(".jpg")]
      random.shuffle(paths)
      cat_total = len(paths)
      train_path.extend(paths[:int(cat_total*0.6)])
      train_y.extend([i]*int(cat_total*0.6))
      val_path.extend(paths[int(cat_total*0.6):int(cat_total*0.8)])
      val_y.extend([i]*len(paths[int(cat_total*0.6):int(cat_total*0.8)]))
      test_path.extend(paths[int(cat_total*0.8):])
      test_y.extend([i]*len(paths[int(cat_total*0.8):]))

  print ("loading training images")
  train_x = read_from_paths(train_path)
  train_x = np.array(train_x, dtype='float16')/255.
  train_y = np.array(train_y)

  print ("loading validation images")
  val_x = read_from_paths(val_path)
  val_x = np.array(val_x, dtype='float16')/255.
  val_y = np.array(val_y)

  print ("loading testing images")
  test_x = read_from_paths(test_path)
  test_x = np.array(test_x, dtype='float16')/255.
  test_y = np.array(test_y)

  #calculate class weight
  classweight = class_weight.compute_class_weight('balanced', np.unique(train_y), train_y)

  #convert to categorical
  train_y = np_utils.to_categorical(train_y, 38)
  val_y = np_utils.to_categorical(val_y, 38)
  test_y = np_utils.to_categorical(test_y, 38)
  print("loaded")

  return train_x,val_x,test_x,train_y,val_y,test_y,classweight,classes

In [37]:
def compiled_two_path_inception_v3(
  classes=1000,
  input_shape=(224,224,3),
  two_paths_first_block=False,
  two_paths_second_block=False,
  max_mix_idx=10):
  base_model = cai.models.two_path_inception_v3(
    include_top=False,
    weights=None, #'two_paths_plant_leafs'
    input_shape=input_shape,
    pooling=None,
    two_paths_first_block=two_paths_first_block,
    two_paths_second_block=two_paths_second_block,
    max_mix_idx=max_mix_idx)
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(38, name='preprediction')(x)
  predictions = Activation('softmax',name='prediction')(x)
  model = Model(inputs=base_model.input, outputs=predictions)
  model.compile(loss='categorical_crossentropy',
  optimizer="sgd",
  metrics=['accuracy','top_k_categorical_accuracy'])
  return model

In [12]:
# test model compilation
model = compiled_two_path_inception_v3(classes=38)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0][0

In [8]:
import os

def load_plantvillage(seed, root_dir):
    if not os.path.exists(root_dir):
        raise FileNotFoundError(f"The directory {root_dir} does not exist.")
        train_x, val_x, test_x, train_y, val_y, test_y, classweight, classes = load_plantvillage(seed=7, root_dir=data_dir)









In [46]:

  find . -name "*.h5"






SyntaxError: invalid syntax (<ipython-input-46-40e25c8f6316>, line 1)

In [41]:

import keras
monitor = 'val_accuracy'
epochs = 30

for max_mix_idx in range(-1, 10, 1):
    basefilename = 'Yosuke-Baseline-V4-' + str(max_mix_idx)
    print('Running: ' + basefilename)

    from keras import Sequential


    def Sequential(classes, input_shape, weights=None):
      model =keras.Sequential(
        classes=38,
        input_shape=(224, 224, 3),
        weights=None
    )



best_result_file_name = basefilename + '-best-result.hdf5'
save_best = keras.callbacks.ModelCheckpoint(
    best_result_file_name,
    monitor='val_acc',
    verbose=1,
    save_best_only=True,
    mode='max'
)
import keras
from keras.models import Model
from keras.layers import Input, Dense


input_layer = Input(shape=(100,))


hidden_layer1 = Dense(64, activation='relu')(input_layer)
hidden_layer2 = Dense(32, activation='relu')(hidden_layer1)


output_layer = Dense(38, activation='softmax')(hidden_layer2)


model = Model(inputs=input_layer, outputs=output_layer)


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

# Define a simple model
model = Sequential([
    Dense(64, activation='relu', input_shape=(100,)),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# Generate placeholder data (replace this with your actual data loading)
train_x = np.random.randn(1000, 100)
train_y = np.random.randint(0, 10, size=(1000,))
val_x = np.random.randn(200, 100)
val_y = np.random.randint(0, 10, size=(200,))
test_x = np.random.randn(200, 100)
test_y = np.random.randint(0, 10, size=(200,))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 32
history = model.fit(
    train_x, train_y,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(val_x, val_y),
    verbose=1  # set verbose=1 for training progress
)

# Evaluate the model
evaluated = model.evaluate(test_x, test_y)
print("Evaluation Results:")
for metric, value in zip(model.metrics_names, evaluated):
    print(f"{metric}: {value}")

# Load the best model (replace 'best_model.h5' with the actual file name)
best_model_file_name = 'best_model.h5'
best_model = keras.models.load_model(best_model_file_name)

# Evaluate the best model
evaluated_best = best_model.evaluate(test_x, test_y)
print("Best Model Evaluation Results:")
for metric, value in zip(best_model.metrics_names, evaluated_best):
    print(f"{metric}: {value}")



print('Best Model Results: ' + basefilename)
model = keras.models.load_model(best_result_file_name, custom_objects={'CopyChannels': cai.layers.CopyChannels})
evaluated = model.evaluate(test_x, test_y)
for metric, name in zip(evaluated, ["loss", "acc", "top 5 acc"]):
    print(name, metric)

print('Finished: ' + basefilename)




Running: Yosuke-Baseline-V4--1
Running: Yosuke-Baseline-V4-0
Running: Yosuke-Baseline-V4-1
Running: Yosuke-Baseline-V4-2
Running: Yosuke-Baseline-V4-3
Running: Yosuke-Baseline-V4-4
Running: Yosuke-Baseline-V4-5
Running: Yosuke-Baseline-V4-6
Running: Yosuke-Baseline-V4-7
Running: Yosuke-Baseline-V4-8
Running: Yosuke-Baseline-V4-9
Epoch 1/10
32/32 [==============================] - 2s 14ms/step - loss: 2.4690 - accuracy: 0.0930 - val_loss: 2.3750 - val_accuracy: 0.1400
Epoch 2/10
32/32 [==============================] - 0s 5ms/step - loss: 2.2643 - accuracy: 0.1550 - val_loss: 2.3558 - val_accuracy: 0.1250
Epoch 3/10
32/32 [==============================] - 0s 4ms/step - loss: 2.1635 - accuracy: 0.1940 - val_loss: 2.3512 - val_accuracy: 0.1250
Epoch 4/10
32/32 [==============================] - 0s 5ms/step - loss: 2.0772 - accuracy: 0.2580 - val_loss: 2.3470 - val_accuracy: 0.1300
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 1.9952 - accuracy: 0.3100 - val_loss

OSError: No file or directory found at best_model.h5